In [8]:
from yelpapi import YelpAPI
import json
import pandas as pd
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

with open('/Users/dusti/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [9]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [10]:
LOCATION = 'Chicago, IL'
TERM = 'Burger'

In [11]:
JSON_FILE = "Data/results_in_progress_Chicago_burger.json"
JSON_FILE

'Data/results_in_progress_Chicago_burger.json'

In [12]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    

    folder = os.path.dirname(JSON_FILE)
    
    if len(folder)>0:
        
        os.makedirs(folder,exist_ok=True)
        
        
    
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Chicago_burger.json not found. Saving empty list to file.


In [13]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [14]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [15]:
total_results = results['total']
total_results

8300

In [16]:
results_per_page = len(results['businesses'])
results_per_page

20

In [17]:
import time, math

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

415

In [18]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [19]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
 
    time.sleep(.2) 

  0%|          | 0/415 [00:00<?, ?it/s]

In [24]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == True:
        if delete_if_exists == True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        folder = os.path.dirname(JSON_FILE)
        if len(folder) > 0:
            os.makedirs(folder, exist_ok=True)
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

In [25]:
create_json_file(JSON_FILE, delete_if_exists=True)

with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

total_results = results['total']

results_per_page = len(results['businesses'])

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Chicago_burger.json already exists. Deleting previous file...
[i] Data/results_in_progress_Chicago_burger.json not found. Saving empty list to new file.
- 0 previous results found.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [26]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
     
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    
    previous_results.extend(results['businesses'])
    
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/415 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [27]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,xoi7Cw7FoknAx5p880RtWQ,au-cheval-chicago,Au Cheval,https://s3-media1.fl.yelpcdn.com/bphoto/ciecqv...,False,https://www.yelp.com/biz/au-cheval-chicago?adj...,8255,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 41.88466, 'longitude': -87.647668}","[pickup, delivery]",$$,"{'address1': '800 W Randolph St', 'address2': ...",+13129294580,(312) 929-4580,9071.234078
1,qxSpiDvad8awYWYZHNoPyQ,nadc-burger-chicago,NADC Burger,https://s3-media1.fl.yelpcdn.com/bphoto/1sRP-X...,False,https://www.yelp.com/biz/nadc-burger-chicago?a...,7,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 41.889635, 'longitude': -87.644829}",[],NaN,"{'address1': '415 N Milwaukee Ave', 'address2'...",,,9610.884910
2,9mhqcimD0CYvqxzfj_VXnQ,little-bad-wolf-chicago,Little Bad Wolf,https://s3-media3.fl.yelpcdn.com/bphoto/aN7fWm...,False,https://www.yelp.com/biz/little-bad-wolf-chica...,2003,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 41.983356, 'longitude': -87.669057}","[pickup, delivery]",$$,"{'address1': '1541 W Bryn Mawr Ave', 'address2...",+17739426399,(773) 942-6399,17601.237441
3,vLNDwB7xaqE5OsmZ0Y3gng,salt-burgers-fries-chicago,Salt Burgers + Fries,https://s3-media2.fl.yelpcdn.com/bphoto/PwsZTc...,False,https://www.yelp.com/biz/salt-burgers-fries-ch...,111,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 41.91072, 'longitude': -87.67592}","[pickup, delivery]",$$,"{'address1': '1920 W North Ave', 'address2': N...",+13127865929,(312) 786-5929,9877.807949
4,7Es7EbTy_s1btAdowLeiPQ,small-cheval-wicker-park-chicago,Small Cheval- Wicker Park,https://s3-media3.fl.yelpcdn.com/bphoto/oyVPVN...,False,https://www.yelp.com/biz/small-cheval-wicker-p...,1362,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 41.9128227233887, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '1732 N Milwaukee Ave', 'address2...",+13128373859,(312) 837-3859,9889.764457


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,-NXMRY4erlpZEfUhzZScYg,bobos-hot-dogs-chicago,Bob-O's Hot Dogs,https://s3-media1.fl.yelpcdn.com/bphoto/g1rZ2t...,False,https://www.yelp.com/biz/bobos-hot-dogs-chicag...,163,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",4.0,"{'latitude': 41.95241, 'longitude': -87.83405}","[pickup, delivery]",$,"{'address1': '8258 W Irving Park Rd', 'address...",+17736259840,(773) 625-9840,15857.253207
996,xrMepV3KUqltASF8VzKE-A,the-glunz-tavern-chicago,The Glunz Tavern,https://s3-media3.fl.yelpcdn.com/bphoto/QyKqY2...,False,https://www.yelp.com/biz/the-glunz-tavern-chic...,152,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 41.90408, 'longitude': -87.63477}","[pickup, delivery]",$$,"{'address1': '1202 N Wells', 'address2': '', '...",+13126423001,(312) 642-3001,11317.245346
997,n47jWHED87LNfosn1Ms5_g,pork-and-mindys-chicago-3,Pork & Mindys,https://s3-media1.fl.yelpcdn.com/bphoto/78y8wH...,False,https://www.yelp.com/biz/pork-and-mindys-chica...,2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.0,"{'latitude': 41.82474233, 'longitude': -87.616...","[pickup, delivery]",NaN,"{'address1': '3857 S Martin Luther King Dr', '...",,,9591.102053
998,V-bg9EhAfwuB2DVGw2gkOw,cody-s-belly-bites-chicago,Cody’s Belly Bites,https://s3-media2.fl.yelpcdn.com/bphoto/TP5nPP...,False,https://www.yelp.com/biz/cody-s-belly-bites-ch...,9,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.809065, 'longitude': -87.643458}","[pickup, delivery]",NaN,"{'address1': '704 W 47th St', 'address2': None...",+17739800014,(773) 980-0014,7715.261833
999,wjACjySYGlQ8_mx8it3FQw,mr-broast-des-plaines,Mr Broast,https://s3-media2.fl.yelpcdn.com/bphoto/-gznsD...,False,https://www.yelp.com/biz/mr-broast-des-plaines...,87,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.5,"{'latitude': 42.0092501, 'longitude': -87.8858...","[pickup, delivery]",$$,"{'address1': '7104 Mannheim Rd', 'address2': '...",+18478135797,(847) 813-5797,23456.454849


In [29]:
final_df.duplicated(subset='id').sum()

2

In [30]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [31]:
final_df.to_csv('Data/final_results_Chicago_burgers.csv.gz', compression='gzip',index=False)